# Построение ML-модели для определения пользователя общих компьютеров

## Задание

Дан датасет (приложенный файл), который описывает действия конкретных пользователей за общими компьютерами в некотором университете. Каждая строчка датасета - это описание рабочей сессии пользователя за компьютером с информацией о том, какие сайты (siteN) и в какое время (timeN) пользователь открыл. Поле target - это метка, идентификатор пользователя.

Ваша задача - построить ML-модель, которая решает задачу определения пользователя по его активности за общими компьютерами.
Можно использовать модели, которые мы изучили на парах (SVM, Logreg, деревья, случайный лес, градиентный бустинг).

Подсказка: данные в чистом виде (названия сайтов и время) загнать в модель не получится. Вам нужно самостоятельно вытащить признаки из этих данных.
Например, из названий сайтов можно вытащить домен, а из времени открытия сайта - время суток, время года и т.д.

## 1. Импорт библиотек и подготовка данных
Прежде всего, нужно разделить данные на признаки и целевую переменную, а затем на тренировочную и тестовую выборки.


In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

# Загрузка файла и просмотр первых строк
file_path = "train_alice.csv"
data = pd.read_csv(file_path).dropna()

# Удаляем ненужные столбцы
features = data.drop(
    columns=["session_id", "target"]
    + [f"time{i}" for i in range(1, 11) if f"time{i}" in data.columns]
)
target = data["target"]

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

## 2. Обучение модели
Попробуем несколько моделей, начиная с простых.


#### Логистическая регрессия:

In [2]:
from sklearn.linear_model import LogisticRegression

# Логистическая регрессия
lr = LogisticRegression(max_iter=1000, class_weight='balanced')
lr.fit(X_train, y_train)

# Оценка модели
y_pred = lr.predict(X_test)
print("Report:\n", classification_report(y_test, y_pred))

Report:
               precision    recall  f1-score   support

           0       0.99      0.29      0.45     41184
           1       0.01      0.76      0.02       410

    accuracy                           0.29     41594
   macro avg       0.50      0.52      0.23     41594
weighted avg       0.98      0.29      0.44     41594



### Дерево решений:


In [3]:
from sklearn.tree import DecisionTreeClassifier

# Дерево решений
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

# Оценка модели
y_pred = tree.predict(X_test)
print("Report (Decision Tree):\n", classification_report(y_test, y_pred))

Report (Decision Tree):
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     41184
           1       0.12      0.16      0.14       410

    accuracy                           0.98     41594
   macro avg       0.56      0.57      0.56     41594
weighted avg       0.98      0.98      0.98     41594



### Случайный лес:

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Случайный лес
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_train, y_train)

# Оценка модели
y_pred = forest.predict(X_test)
print("Report (Random Forest):\n", classification_report(y_test, y_pred))

Report (Random Forest):
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     41184
           1       0.79      0.04      0.07       410

    accuracy                           0.99     41594
   macro avg       0.89      0.52      0.53     41594
weighted avg       0.99      0.99      0.99     41594



### Градиентный бустинг:

In [5]:
from sklearn.ensemble import GradientBoostingClassifier

# Градиентный бустинг
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb.fit(X_train, y_train)

# Оценка модели
y_pred = gb.predict(X_test)
print("Report (Gradient Boosting):\n", classification_report(y_test, y_pred))

Report (Gradient Boosting):
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     41184
           1       0.89      0.14      0.24       410

    accuracy                           0.99     41594
   macro avg       0.94      0.57      0.62     41594
weighted avg       0.99      0.99      0.99     41594



### SVM (метод опорных векторов):


In [6]:
from sklearn.svm import SVC

# Слабый комп, поэтому
X_train_sample = X_train.sample(n=200, random_state=42)
y_train_sample = y_train[X_train_sample.index]

# Метод опорных векторов
svc = SVC(kernel='linear')
svc.fit(X_train_sample, y_train_sample)

# Оценка модели
y_pred = svc.predict(X_test)
print("Report (SVM):\n", classification_report(y_test, y_pred))

Report (SVM):
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     41184
           1       0.01      0.01      0.01       410

    accuracy                           0.98     41594
   macro avg       0.50      0.50      0.50     41594
weighted avg       0.98      0.98      0.98     41594

